![atlas](atlas4.png)

A fire atlas is a configuration convention for geospatial assets related to community fire planning and response together with a configuration for ways to instantiate, edit, and manage those assets.

A stewardship atlas is a data set, a confuration for storing, processing, and sharing that data set, and a set of implementions to do so.

In [1]:
# Boring Imports
import sys, os, subprocess, time, json, string, datetime, random, math
sys.path.insert(0, "/root/stewardship_atlas/python")

# our Imports
import json_config
import dataswale_geojson
import deltas_geojson as deltas
import versioning
import vector_inlets
import raster_inlets
import outlets
import atlas
import utils
import eddies


/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


# Create a new atlas from GeoJSON

In [2]:
gj = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
          "name": "mineralkinsey", 
          "logo": "https://img1.wsimg.com/isteam/ip/11ac4488-96b5-4627-b850-ebbe7cdd6cf8/logo.png"},
      "geometry": {
        "coordinates": [ utils.bbox_to_polygon({
            "south": 40.18,
            "north": 40.21,
            "west": -123.95,
            "east": -123.9
            
        })],
        "type": "Polygon"
      }}]}

layers = json.load(open("../configuration/MineralKinsey_layers.json"))
assets = json.load(open("../configuration/MineralKinsey_assets.json"))

c = atlas.create(feature_collection=gj, layers=layers, assets=assets)

Updating password for user internal
Updating password for user admin
Updating password for user admin
Updating password for user internal
Updating password for user admin
Updating password for user admin
Updating password for user admin
Updating password for user admin
Updating password for user admin
Updating password for user admin
2025-08-25 23:34:29,337 - atlas - INFO - Storing initial feature collection in {p /'layers' / 'regions'}...
2025-08-25 23:34:29,339 - atlas - INFO - built a config for mineralkinsey.


### Or use an existing swale and config:

In [ ]:
c = json.load(open("/root/swales/scvfd/staging/atlas_config.json"))


# New School

### Elevation and Hillshade rasters and vectors

In [ ]:
# Lidar hillshade
raster_inlets.local_raster(config=c, name="local_hillshade", delta_queue=deltas)
dataswale_geojson.refresh_raster_layer(c, 'lidar_basemap', deltas.apply_deltas)

In [ ]:
# Still need DEM for contours - make `elevation` layer, then eddy maps:
# * derived_hillshade: elevation -> basemap 
# * gdal_contours: elevation -> contours
raster_inlets.url_raster(config=c, name="dem", delta_queue=deltas)
dataswale_geojson.refresh_raster_layer(c, 'elevation', deltas.apply_deltas)

In [ ]:
dataswale_geojson.eddy(c, 'gdal_contours')

In [ ]:
dataswale_geojson.eddy(c, 'derived_hillshade')

### Core vector layers

In [ ]:
# vector_inlets.overture_duckdb(config=c, name="public_roads", delta_queue=deltas)
atlas.materialize(materializers, c, 'public_roads')
dataswale_geojson.refresh_vector_layer(c, 'roads', delta_queue_builder=deltas.apply_deltas_overwrite)

In [ ]:
# vector_inlets.local_ogr(config=c, name="public_creeks", delta_queue=deltas)
atlas.materialize(materializers, c, 'public_creeks')

dataswale_geojson.refresh_vector_layer(c, 'creeks', delta_queue_builder=deltas.apply_deltas_overwrite)

In [ ]:
c['assets'].keys()

In [ ]:
# vector_inlets.local_ogr(config=c, name="local_milemarkers", delta_queue=deltas)
atlas.materialize(materializers, c, 'local_burns')

In [ ]:
dataswale_geojson.refresh_vector_layer(c, 'burns', deltas.apply_deltas)

### Buildings, Parcels, and Addresses

In [ ]:
vector_inlets.overture_duckdb(config=c, name="public_buildings", delta_queue=deltas)

dataswale_geojson.refresh_vector_layer(c, 'buildings', delta_queue_builder=deltas.apply_deltas_overwrite)

#### First load parcel layer from OpenAddresses

In [ ]:
vector_inlets.local_ogr(config=c, name="oa_parcels", delta_queue=deltas)

dataswale_geojson.refresh_vector_layer(c, 'parcels', deltas.apply_deltas_overwrite)

#### Then get addresses from Oveture - as a Delta to annotate parcels

In [ ]:
vector_inlets.overture_duckdb(config=c, name="public_addresses", delta_queue=deltas)

In [ ]:
eddies.delta_annotate_spatial_duckdb(c, "parcels", "public_addresses__20250703_161745__create")

In [ ]:
# No need to run this since the above annoation updates the layer
dataswale_geojson.refresh_vector_layer(c, 'parcels', deltas.apply_deltas)

#### Now use parcel layer as annotation for Buildings

In [ ]:
eddies.delta_annotate_spatial_duckdb(c, "buildings", "parcels", anno_type="layers")

### POI etc

In [ ]:
atlas.materialize(c, 'public_poi')

In [ ]:
dataswale_geojson.refresh_vector_layer(c, 'natural')

In [ ]:
vector_inlets.local_ogr(config=c, name="local_ponds", delta_queue=deltas)

In [ ]:
dataswale_geojson.refresh_vector_layer(c, 'ponds', deltas.apply_deltas)

In [ ]:
vector_inlets.local_ogr(config=c, name="local_hydrants", delta_queue=deltas)

In [ ]:
dataswale_geojson.refresh_vector_layer(c, 'hydrants', deltas.apply_deltas)

In [ ]:
vector_inlets.local_ogr(config=c, name="local_creeks", delta_queue=deltas)

In [ ]:
dataswale_geojson.refresh_vector_layer(c, 'creeks', deltas.apply_deltas)

In [ ]:
vector_inlets.local_ogr(config=c, name="local_helilandings", delta_queue=deltas)

In [ ]:
dataswale_geojson.refresh_vector_layer(c, 'helilandings', deltas.apply_deltas)

### outlets

In [ ]:
outlets.outlet_html(c, 'html')


In [ ]:
outlets.outlet_webmap(c, 'webmap')

In [3]:
eddies.h3_cells(c, 'burns', 'burns')

eddies - INFO - Processing H3 cells for layer 'burns' with resolution 11, algorithm 'max_num_cells', max_cells 10
eddies - INFO - Layer 'burns' has geometry type: polygon
eddies - INFO - Processing 28 features in layer 'burns'
eddies - INFO - Hape conversion: [[40.196547, -123.914508], [40.196608, -123.914072], [40.196784, -123.913747], [40.196939, -123.913526], [40.197193, -123.913255], [40.196983, -123.912886], [40.196823, -123.91245], [40.196784, -123.912373], [40.196674, -123.912102], [40.19663, -123.911992], [40.196569, -123.911898], [40.196492, -123.911843], [40.196431, -123.911815], [40.196503, -123.912814], [40.196415, -123.913311], [40.195951, -123.913807], [40.195841, -123.913901], [40.195582, -123.914552], [40.195124, -123.915291], [40.194997, -123.915656], [40.19503, -123.915838], [40.195157, -123.915926], [40.195339, -123.915942], [40.195466, -123.91586], [40.19562, -123.915733], [40.196547, -123.914508]] -> ['8b28062ca44efff', '8b28062ca44afff', '8b28062ca734fff', '8b2806

eddies - INFO - Hape conversion: [[40.196895, -123.91117], [40.197215, -123.910789], [40.197347, -123.910651], [40.197601, -123.910773], [40.19775, -123.911197], [40.197943, -123.911468], [40.198291, -123.911733], [40.198484, -123.911986], [40.198743, -123.912279], [40.198848, -123.912472], [40.198876, -123.912676], [40.198765, -123.913035], [40.198616, -123.913233], [40.198473, -123.913438], [40.198296, -123.913625], [40.198186, -123.913708], [40.198081, -123.913758], [40.197993, -123.913769], [40.197844, -123.913603], [40.197733, -123.913443], [40.19764, -123.913377], [40.197491, -123.913344], [40.19732, -123.913289], [40.197226, -123.913129], [40.19716, -123.91298], [40.197104, -123.912891], [40.197016, -123.91282], [40.197038, -123.912671], [40.197099, -123.911771], [40.196895, -123.91117]] -> ['8a28062ca7b7fff', '8a28062ca7a7fff', '8a28062ca45ffff', '8a28062ca4effff']
eddies - INFO - Indexing Geom (Swap: True): {"coordinates": [[[-123.91117, 40.196895], [-123.910789, 40.197215], [

eddies - INFO - Hape conversion: [[40.204037, -123.908334], [40.203714, -123.908317], [40.203317, -123.908541], [40.202986, -123.908723], [40.202804, -123.908839], [40.202614, -123.908781], [40.202349, -123.908681], [40.202208, -123.908657], [40.202117, -123.908624], [40.202183, -123.90917], [40.202109, -123.909675], [40.201918, -123.910006], [40.201976, -123.910477], [40.202109, -123.911065], [40.202167, -123.911272], [40.202316, -123.911413], [40.202531, -123.911437], [40.202787, -123.911272], [40.20316, -123.910941], [40.203342, -123.910825], [40.203499, -123.910717], [40.203574, -123.91061], [40.203847, -123.910006], [40.203954, -123.909642], [40.204037, -123.909385], [40.204095, -123.908988], [40.204037, -123.908334]] -> ['8a28062d9a67fff', '8a28062d9a77fff']
eddies - INFO - Indexing Geom (Swap: True): {"coordinates": [[[-123.908334, 40.204037], [-123.908317, 40.203714], [-123.908541, 40.203317], [-123.908723, 40.202986], [-123.908839, 40.202804], [-123.908781, 40.202614], [-123.9

eddies - INFO - Hape conversion: [[40.200561, -123.902408], [40.201289, -123.901961], [40.201786, -123.901961], [40.20215, -123.902507], [40.201968, -123.9036], [40.201918, -123.904527], [40.201819, -123.905123], [40.20172, -123.905967], [40.201654, -123.906943], [40.201587, -123.90749], [40.201587, -123.90787], [40.201571, -123.908417], [40.201505, -123.908781], [40.201223, -123.908897], [40.200843, -123.908847], [40.200694, -123.908483], [40.200313, -123.908367], [40.200098, -123.908417], [40.199932, -123.908466], [40.199684, -123.908549], [40.199402, -123.908532], [40.199022, -123.908384], [40.19884, -123.908102], [40.198724, -123.907837], [40.198525, -123.907606], [40.19836, -123.907539], [40.199469, -123.906066], [40.200048, -123.905536], [40.200462, -123.904709], [40.200909, -123.904063], [40.201008, -123.903567], [40.200958, -123.903153], [40.200809, -123.902805], [40.200561, -123.902408]] -> ['8a28062d9b87fff', '8a28062d9b0ffff', '8a28062d9b07fff', '8a28062d9a27fff', '8a28062d9

eddies - INFO - Indexing Geom (Swap: True): {"coordinates": [[[-123.925751, 40.19197], [-123.924634, 40.191623], [-123.924286, 40.190977], [-123.923889, 40.190357], [-123.923591, 40.191623], [-123.923095, 40.192268], [-123.922176, 40.192616], [-123.922797, 40.193386], [-123.923517, 40.194031], [-123.924187, 40.19423], [-123.924684, 40.193982], [-123.925329, 40.193361], [-123.925602, 40.192715], [-123.925851, 40.192194], [-123.925751, 40.19197]]], "type": "Polygon"} got {'cells': ['8a28062ca06ffff', '8a28062caa97fff', '8a28062ca067fff', '8a28062ca047fff', '8a28062ca14ffff'], 'resolution': 10, 'cell_count': 5, 'representative_index': '8a28062ca06ffff'}
eddies - DEBUG - Feature 13: Added H3 properties with 5 cells at resolution 10
eddies - INFO - Hape conversion: [[40.194404, -123.92369], [40.194006, -123.922821], [40.193038, -123.922027], [40.193634, -123.921505], [40.193559, -123.92081], [40.193907, -123.920214], [40.194329, -123.92019], [40.194826, -123.920388], [40.195148, -123.920115

eddies - DEBUG - Feature 17: Added H3 properties with 9 cells at resolution 11
eddies - INFO - Hape conversion: [[40.199183, -123.910928], [40.198898, -123.910804], [40.198562, -123.910966], [40.198327, -123.911226], [40.198227, -123.911326], [40.198103, -123.911363], [40.198066, -123.911301], [40.198115, -123.911028], [40.198252, -123.910854], [40.198302, -123.910581], [40.198327, -123.91032], [40.198202, -123.910109], [40.198078, -123.909948], [40.197991, -123.909687], [40.198004, -123.909339], [40.198153, -123.90893], [40.198364, -123.908781], [40.198761, -123.909464], [40.199009, -123.909886], [40.199121, -123.910084], [40.199171, -123.910233], [40.199171, -123.910308], [40.199183, -123.910928]] -> ['8b28062ca4c8fff', '8b28062ca4cefff', '8b28062ca4cbfff', '8b28062ca4cafff', '8b28062ca4cdfff', '8b28062ca4c3fff']
eddies - INFO - Indexing Geom (Swap: True): {"coordinates": [[[-123.910928, 40.199183], [-123.910804, 40.198898], [-123.910966, 40.198562], [-123.911226, 40.198327], [-123.9

eddies - INFO - Indexing Geom (Swap: True): {"coordinates": [[[-123.911121, 40.19688], [-123.91086, 40.196669], [-123.910745, 40.196514], [-123.910658, 40.197045], [-123.910587, 40.197166], [-123.910587, 40.197278], [-123.910627, 40.197327], [-123.911093, 40.196917], [-123.911121, 40.19688]]], "type": "Polygon"} got {'cells': ['8b28062ca4ecfff'], 'resolution': 11, 'cell_count': 1, 'representative_index': '8b28062ca4ecfff'}
eddies - DEBUG - Feature 21: Added H3 properties with 1 cells at resolution 11
eddies - INFO - Hape conversion: [[40.196902, -123.908278], [40.196613, -123.908017], [40.196371, -123.907785], [40.196232, -123.907626], [40.19612, -123.907468], [40.195999, -123.907226], [40.19585, -123.907384], [40.195654, -123.90771], [40.195505, -123.907896], [40.195347, -123.908157], [40.195254, -123.908259], [40.195245, -123.908408], [40.195263, -123.908567], [40.195403, -123.908632], [40.195477, -123.908799], [40.195598, -123.908883], [40.19585, -123.908967], [40.196166, -123.90879

eddies - INFO - Hape conversion: [[40.191316, -123.926496], [40.190558, -123.925838], [40.19026, -123.925491], [40.190186, -123.925404], [40.190447, -123.925006], [40.190409, -123.924684], [40.190335, -123.924237], [40.190273, -123.924013], [40.190273, -123.923926], [40.191489, -123.924634], [40.191651, -123.924957], [40.191924, -123.925788], [40.191961, -123.925975], [40.191316, -123.926496]] -> ['8a28062caab7fff']
eddies - INFO - Indexing Geom (Swap: True): {"coordinates": [[[-123.926496, 40.191316], [-123.925838, 40.190558], [-123.925491, 40.19026], [-123.925404, 40.190186], [-123.925006, 40.190447], [-123.924684, 40.190409], [-123.924237, 40.190335], [-123.924013, 40.190273], [-123.923926, 40.190273], [-123.924634, 40.191489], [-123.924957, 40.191651], [-123.925788, 40.191924], [-123.925975, 40.191961], [-123.926496, 40.191316]]], "type": "Polygon"} got {'cells': ['8a28062caab7fff'], 'resolution': 10, 'cell_count': 1, 'representative_index': '8a28062caab7fff'}
eddies - DEBUG - Feat

Exception: H3 cells eddy failed: module 'utils' has no attribute 'save_layer'

# Messing About

In [ ]:
import geojson
import dataswale_geojson as dataswale
p = geojson.GeoJSON()
bc = dataswale.layer_as_featurecollection(c, 'burns')

In [ ]:
import h3 
pc=[[b,a] for a,b in bc[0]['geometry']['coordinates'][0]]
hs=h3.LatLngPoly(pc)   

In [ ]:
h3.h3shape_to_cells(hs,11)

In [ ]:
versioning.publish_new_version(c)

In [ ]:
import gspread
gc = gspread.service_account()

In [ ]:

ss = gc.open("scvfd Fire Atlas: poi")
wks = ss.sheet1

In [ ]:
ss.url

In [ ]:
features = wks.get_all_records()
features

In [ ]:
s="""{"version_string": "staging", "versions": ["published"], "logo": "/local/scs-smallgrass1.png", "swaleName": "westport", "consoleType": "INTERNAL", "interfaces": [{"type": "outlet", "in_layers": ["basemap", "parcels", "contours", "roads", "internal_roads", "turnouts", "milemarkers", "creeks", "buildings", "helilandings", "hydran\
ts"], "config_def": "webmap", "access": ["internal", "admin"], "config": {"asset_type": "outlet", "name": "webmap", "fetch_type": "webmap", "interaction": "interface", "attribu\
tion": {"url": "https://portal.opentopography.org/api/terms", "description": "Open Topography's Global DEM", "license": "https://portal.opentopography.org/api/terms", "citation\
": "https://portal.opentopography.org/api/terms"}}}, {"type": "outlet", "name": "runbook", "in_layers": ["lidar_basemap", "tsunami", "contours", "roads", "creeks", "buildings",\
 "campgrounds", "helilandings", "milemarkers", "hydrants"], "access": ["internal", "admin"], "config_def": "runbook", "regions": [{"bbox": {"east": -121.19978063408502, "west":\
 -121.20391109005016, "south": 39.23863749098538, "north": 39.24416744687048}, "name": "RockLoop", "caption": "Double loop with Ginger and big rocks.", "vectors": [], "raster":\
 "", "text": "Hwy20 to the County Lnie"}, {"bbox": {"east": -121.19978063408502, "west": -121.20391109005016, "south": 39.23863749098538, "north": 39.24416744687048}, "name": "\
GolfCourses", "caption": "Golf Courses", "vectors": [], "raster": "", "text": "Golf Courses"}], "config": {"asset_type": "outlet", "layers": ["hillshade", "contours", "creeks",\
 "tsunami", "buildings", "helilanding", "overture_roads"], "fetch_type": "runbook", "interaction": "interface", "data_type": "html", "outpath_template": "/runbook/{name}_page_{\
i}.html", "page_name_template": "{name}_page_{i}", "attribution": {"url": "https://www.scvfd.fireatlas.org", "description": "Runbook of the atlas, a linked set of specific subm\
ap/diagram pages.", "license": "https://www.scvfd.fireatlas.org", "citation": "https://www.scvfd.fireatlas.org"}}}], "downloads": [], "useCases": [{"name": "Firefighter", "case\
s": ["Download Avenza version", "Share a QR Code for Avenza", "Mark an Incident", "Mark a POI"]}, {"name": "GIS Practitioner", "cases": ["Download Layer GeoJSON", "Download Geo\
PKG", "Add a layer as GeoJSON"]}, {"name": "Administrator", "cases": ["Go to Admin interface", "Switch Version"]}], "layers": []}"""
print(json.dumps(json.loads(s)['interfaces'], indent=2))

In [ ]:
[].count('d')

In [ ]:
def canonicalize_name(s):
    return "_".join(s.lower().split()).strip()

In [ ]:
canonicalize_name("    h    IU H8  hdh")

In [ ]:
#json.dump(hmm, open("newregions.json", "w"))
regions = json.load( open("newregions.json"))

In [ ]:
utils.geojson_to_bbox([
            [
              -121.19978063408502,
              39.24416744687048
            ],
            [
              -121.20391109005016,
              39.24416744687048
            ],
            [
              -121.20391109005016,
              39.23863749098538
            ],
            [
              -121.19978063408502,
              39.23863749098538
            ],
            [
              -121.19978063408502,
              39.24416744687048
            ]
          ])

In [ ]:
regions[9]['vectors'][-1][0]=ac['assets']['overture_roads']


In [ ]:
atlas_outlets.build_region_map('/root/data/', 'westport', 'stage', 'runbook', regions[9])

In [ ]:
"DFDf".lower()

In [ ]:
hmm = atlas.asset_materialize(ac, dc, ac['assets']['webmap_public'])

In [ ]:
z=json.load(open("zones.geojson"))

In [ ]:
z

In [ ]:
ac['assets'].keys()

In [ ]:
regions[1]['vectors'][0][0]['color']=[0,255,0]

In [ ]:
del(regions[1]['vectors'][0])

In [ ]:
regions[0][1]['raster'] = [ac['assets']['hillshade'], '/root/data/scvfd/stage/staging/hillshade_1_A.tiff']

In [ ]:
atlas_outlets.build_region_map('/root/data/', 'westport', 'stage', '', regions[1])

In [ ]:
#ht5#.format(ea=ac['assets']['creeky'],**ac['assets']['creeky'])
ht1.format(name=ac['assets']['creeky']['name'])
#ac['assets']['creeky']

In [ ]:
import planetary_computer

In [ ]:
dir(planetary_computer)

In [ ]:
for i in range(10) if i %% 3:
    print(i)
    

In [ ]:
j = json.load(open('/root/data/scvfd/staging/roads.geojson'))

from collections import Counter
c=Counter()
for f in j['features']:
    
    c.update(f['properties'].keys())

regions = atlas.asset_materialize(ac, dc, ac['assets']['gazetteer'])

import atlas_outlets
atlas_outlets.export_region_layer_raster('/root/data/', 'scvfd', 'hillshade', regions[0])

In [ ]:
outlets.sql_query(c, 'sqldb', 
                  f"""SELECT column_name 
                  FROM information_schema.columns 
                  WHERE table_name = 'creeks';""")

In [ ]:
regions = atlas_outlets.generate_regions(dc, ac['assets']['gazetteer'])

region_json = json.dump(regions,open('regions_scvfd.json', 'w'))

import atlas_eddies
atlas_eddies.generate_contours('/root/data/scvfd/stage/staging/dem.tiff', '/root/data/scvfd/stage/staging/')

# Queries and DuckDB

In [ ]:
import duckdb
duckdb.sql("""                                                                                                                   
INSTALL spatial;                                                                                                                     
LOAD spatial;                                                                                                                        
""")


In [ ]:
#duckdb.sql("SELECT * FROM duckdb_settings()")
duckdb.sql("SHOW TABLES")

In [ ]:
duckdb.sql("SET s3_region='us-west-2'; DROP TABLE IF EXISTS addresses; CREATE TABLE addresses AS SELECT  * EXCLUDE geometry, ST_AsGeoJSON(ST_GeomFromWKB(geometry)) as geom FROM   read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=addresses/type=*/*', filename=true, hive_partitioning=1) WHERE  bbox.xmin > -123.85333388805361  AND bbox.xmax < -123.67763908625702  AND bbox.ymin > 39.53857292439969  AND bbox.ymax < 39.89595306214338  AND street != 'NONE';")

In [ ]:
# (min_x DOUBLE, min_y DOUBLE, ma
obq = """
SET s3_region='us-west-2';

SELECT
  * EXCLUDE geometry, -- ST_GeomFromWKB(geometry) as geom,
FROM
  read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=buildings/type=building/*', filename=true, hive_partitioning=1)
WHERE
  bbox.xmin > {west}
  AND bbox.xmax < {east}
  AND bbox.ymin > {south}
  AND bbox.ymax < {north}
;
""".format(**ac['dataswales'][0]['geometry']['bbox'])
address_query

In [ ]:
# (min_x DOUBLE, min_y DOUBLE, ma
address_query = """
SET s3_region='us-west-2';
DROP TABLE IF EXISTS adds;
CREATE TABLE adds AS 
SELECT
  * EXCLUDE geometry, -- ST_GeomFromWKB(geometry) as geom,
  ST_AsGeoJSON(ST_MakeEnvelope(bbox.xmin-0.0002, bbox.ymin-0.0002, bbox.xmax+0.0002, bbox.ymax+0.0002)) as geom
FROM
  read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=addresses/type=*/*', filename=true, hive_partitioning=1)
WHERE
  bbox.xmin > {west}
  AND bbox.xmax < {east}
  AND bbox.ymin > {south}
  AND bbox.ymax < {north}
  AND street != 'NONE';
""".format(**ac['dataswales'][0]['geometry']['bbox'])
address_query

In [ ]:
# (min_x DOUBLE, min_y DOUBLE, ma
roads_query = """
SET s3_region='us-west-2';
DROP TABLE IF EXISTS adds;
CREATE TABLE overture_roads AS 
SELECT
  * EXCLUDE geometry, ST_AsGeoJSON(ST_GeomFromWKB(geometry)) as geometry
FROM
  read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=transportation/type=segment/*', filename=true, hive_partitioning=1)
WHERE
  bbox.xmin > {west}
  AND bbox.xmax < {east}
  AND bbox.ymin > {south}
  AND bbox.ymax < {north}
  
""".format(**ac['dataswales'][0]['geometry']['bbox'])
roads_query

In [ ]:
res = duckdb.sql(obq)

In [ ]:
len(res)

In [ ]:
for row in res.fetchall():
    

In [ ]:
duckdb.sql("""select class,subclass FROM adds WHERE names.primary LIKE '%Usal%';""")

In [ ]:
duckdb.sql("select class, subclass, road_surface[1]['value'] , count(*)  from adds group by (class,subclass, road_surface[1]['value'] ) ORDER BY count(*) DESC;").fetchall()

In [ ]:
duckdb.sql("DESCRIBE adds;")

In [ ]:
annos_path = '/root/data/westport/stage/staging/addresses.geojson'
duckdb.sql("DROP TABLE IF EXISTS annos;")
duckdb.sql(f"CREATE TABLE annos AS SELECT COLUMNS('.*') AS \"annos_\\0\"  FROM ST_Read('{annos_path}');")

feat_path = '/root/data/westport/stage/staging/raw_buildings.geojson'
duckdb.sql("DROP TABLE IF EXISTS features;")
duckdb.sql(f"CREATE TABLE features AS SELECT COLUMNS('.*') AS \"features_\\0\"  FROM ST_Read('{feat_path}');")

In [ ]:
missed_sql="""
SELECT  features.* EXCLUDE features_geom, ST_AsGeoJSON(features_geom) AS geometry                                                
    FROM features LEFT JOIN annos                                                                                                    
    ON ST_Intersects(annos_geom, features_geom)                                                                                      
    WHERE annos_geom IS NULL; 
    """

In [ ]:
mr=duckdb.sql(missed_sql)

In [ ]:
mr.columns

In [ ]:
duckdb.sql("CREATE TABLE buildings AS SELECT * FROM ST_Read('/root/data/westport/stage/staging/buildings.geojson');")

In [ ]:
r=duckdb.sql(
    "SELECT *   FROM adds JOIN buildings ON ST_Intersects(adds.geom, buildings.geom);")

In [ ]:
r.fetchone()

In [ ]:
duckdb.sql(f"DROP TABLE annos")
duckdb.sql(f"CREATE TABLE annos AS SELECT COLUMNS('.*') AS \"annos_\\0\" FROM ST_Read('/root/data/scvfd/stage/ponds_anno/data_20250414_005330.json');")
duckdb.sql(f"DROP TABLE features")
duckdb.sql(f"CREATE TABLE features AS SELECT  COLUMNS('.*') AS \"features_\\0\"  FROM ST_Read('/root/data/scvfd/stage/staging/ponds.geojson');")

In [ ]:
geojoin="""
with foo AS (
    SELECT  *,ST_Distance(ST_Centroid(features_geom), ST_CEntroid(annos_geom)) as dist
    FROM features JOIN annos 
    ON ST_Intersects(annos_geom, features_geom)
    ) 
SELECT b.* EXCLUDE features_geom, ST_AsGeoJSON(b.features_geom) AS geom FROM foo AS b LEFT JOIN foo as c 
ON b.features_geom == c.features_geom AND b.dist < c.dist
WHERE c.dist IS NULL;
"""

In [ ]:
r2=duckdb.sql(geojoin)

In [ ]:
w=r2.fetchall()

In [ ]:
r2.columns

In [ ]:
r2.columns

In [ ]:
r2.columns

In [ ]:
duckdb.sql("describe table features")

In [ ]:
r=duckdb.sql(
    "SELECT * EXCLUDE (annos_geom,features_geom),ST_AsGeoJSON(features_geom) AS geometry  FROM features LEFT JOIN annos ON ST_Intersects(annos_geom, features_geom);")

In [ ]:
res=[dict(zip(r.columns, row)) for row in r.fetchall()]

In [ ]:
res

In [ ]:
fs = geojson.FeatureCollection([])
#dir(fs)

In [ ]:
features = []
for mf in res[:3]:
    f = geojson.Feature()
    for k,v	in ( (k[9:],v) for k,v in mf.items() if k.startswith('features_')):
       
        if k == 'geom':
            f['geometry'] = v
        else:
            f['properties'][k] = v
    features.append(f)
fs = geojson.FeatureCollection(features)

In [ ]:
geojson.dumps(fs, sort_keys=True)

In [ ]:
s=huh[0]['geometry']

In [ ]:
dir(duckdb)

In [ ]:
geojson.loads(s)

In [ ]:
fc = geojson.FeatureCollection()
for f in res:
    fr = geojson.Feature()
    for k,v in fr.items():
    geometry=f['features_geometry'])
    
    del(f['features_geometry'])
    del(f['annos_geometry'])
    f['properties'] = 
    

# WVFD adaptation

In [ ]:
def points_to_bbox(points):
    xset = [p[0] for p in points]
    yset = [p[1] for p in points]
    return {
        "north" : max(yset),
        "south" : min(yset),
        "east": max(xset),
        "west": min(xset),
        #"xset": xset,
        #"yset": yset
    }

roi = json.load(open("westport_runbook.geojson"))
[{'bbox': points_to_bbox(f['geometry']['coordinates'][0]),
    "name": f['properties']['Description'],
    "layers": ["hillshade", "buildings", "creeky", "roads"],
    "caption": f['properties']['Description'],
    "vectors": [], "raster": "",
     "text": f['properties']['Description'] } for f in roi['features']]

In [ ]:
roi['features'][1]

In [ ]:
#roi['features'][1]['geometry']['coordinates']
points_to_bbox(roi['features'][1]['geometry']['coordinates'][0])
points_to_bbox(roi['features'][1]['geometry']['coordinates'][0])

In [ ]:
import geopandas
geo = dc['geometry']['bbox']
geo_tuple = (geo['west'], geo['south'], geo['east'], geo['north'])
gpd = geopandas.read_file("/root/data/WVFD_clippedTA83roads2025_02_04.gpkg",
                        ).to_crs(crs=dc['crs']).clip(geo_tuple)

# Prototyping for tranlations and adaptors

In [ ]:
heli_str = """
39.3342	123.4594	
39.3325	123.4588	
		
39.3444	123.4635	
39.3414	123.4626	
		
39.3681	123.4689	Power Lines overhead
39.3595	123.4708	
39.3829	123.4714	
39.385	123.471	
39.3829	123.4714	
39.3898	123.468	
39.3944	123.4707	
39.3964	123.4709	
39.3944	123.4747	
39.396	123.467	
39.3925	123.4651	
39.7979	123.4617	
39.4074	123.4736	
39.4044	123.4739	
39.3944	123.4707	
39.3925	123.4651	616 ft ABSL
39.4044	123.4739	
39.4022	123.4677	Powerlines South of Road
39.4116	123.4756	
39.4074	123.4736	
39.4214	123.4823	Closed in Winter
39.4237	123.4825	
39.4425	123.4888	Rockport
39.4647	123.4994	
39.4503	123.4909	"""
helis=[l.split(maxsplit=2) for l in heli_str.split("\n")]

In [ ]:
def hack(s):
    deg, rest = s.split('.')
    degs = float(deg)
    badmins = float(rest)/100.0
    mins = float( rest[:2] + '.' + rest[2:])
    #secs = float(rest[2:])
    
    fdeg = degs + mins/60.0 
    #print(f"D: {degs} M: {mins} NOT {badmins} -> {fdeg}")
    return fdeg#+ secs/3600.0

hack('123.471')

In [ ]:
import geojson
def tuples_to_features(tuples):
    features = []
    for t in tuples:
        if not t:
            continue
        print(f"({t[0]}, -{t[1]}) -> (-{hack(t[1])}, {hack(t[0])})")
        f = geojson.Feature(
            geometry=geojson.Point( (-1*hack(t[1]), hack(t[0]) ) ),
            properties = {'name': t[2]} if len(t) > 2 else {}
        )
        features.append(f)
    return geojson.FeatureCollection(features)

def tsv_to_geojson(rawstring, outpath, as_string=False):
    tuples = [l.split(maxsplit=2) for l in rawstring.split("\n")]
    fc = tuples_to_features(tuples)
    if as_string:
        return geojson.dumps(fc)
    else:
        return geojson.dump(fc, open(outpath,"w"))
    

In [ ]:
tsv_to_geojson(heli_str, 'helilandings.geojson', as_string=False)

# Attic

In [ ]:
[(r['bbox']['north'] , r['bbox']['south']) for r in regions]

In [ ]:
c = json.load(open("/root/data/scvfd/staging/creeks.geojson"))
from collections import Counter
Counter([f['properties']['fcode_description'] for f in c['features']])

In [ ]:
ht1="""
            <!DOCTYPE html>

            <html>
            <head>
                <meta charset="utf-8">
                <title>Edit {name}</title>
                <script src='https://unpkg.com/maplibre-gl@3.6.2/dist/maplibre-gl.js'></script>
                <script src="https://unpkg.com/terra-draw@1.0.0-beta.0/dist/terra-draw.umd.js"></script>
                <link href='https://unpkg.com/maplibre-gl@3.6.2/dist/maplibre-gl.css' rel='stylesheet' />

                <style>
                    body {{ margin: 0; padding: 0; }}
                    #map {{ position: absolute; top: 0; bottom: 0; width: 100%; }}
                    #controls {{
                        position: absolute;
                        top: 10px;
                        right: 10px;
                        z-index: 1;
                        background: white;
                        padding: 10px;
                        border: 1px solid #ccc;
                        border-radius: 4px;
                    }}
                    #feature-text {{
                        margin-bottom: 10px;
                        padding: 5px;
                        width: 200px;
                    }}
                    #save-button {{
                        padding: 10px;
                        background: white;
                        border: 1px solid #ccc;
                        border-radius: 4px;
                        cursor: pointer;
                        width: 100%;
                    }}
                    #save-button:hover {{
                        background: #f0f0f0;
                    }}
                </style>
            </head>
            <body>
            <center>
            <h1>Edit Note Layer: {name}</h1>
            <i>Click to draw, double click to end, click "Save" button when done.</i>
            
            <A HREF="../html">cancel</a>
            </center>
            <div id="controls">
                    <input type="text" id="feature-name" placeholder="Enter name" />
                    
                    <input type="radio" id="feature-fcode_description" name="feature-fcode_description" value="Stream/River: Hydrographic Category = Intermittent" />
                    
            <button id="save-button">Save Note</button>
            </div>
            
            <div id="map"></div>
            <script>
            """
ht2="""
                var map = new maplibregl.Map({
  "container": "map",
  "style": {
    "glyphs": "https://fonts.undpgeohub.org/fonts/{fontstack}/{range}.pbf",
    "version": 8,
    "center": [
      -123.90625574273562,
      40.211
    ],
    "zoom": 12,
    "sources": {
      "hillshade": {
        "type": "image",
        "url": "../staging/hillshade.tiff.jpg",
        "coordinates": [
          [
            -123.98,
            40.26
          ],
          [
            -123.83251148547124,
            40.26
          ],
          [
            -123.83251148547124,
            40.162
          ],
          [
            -123.98,
            40.162
          ]
        ]
      },
      "roads": {
        "type": "geojson",
        "data": "/scvfd/staging/roads.geojson"
      },
      "buildings": {
        "type": "geojson",
        "data": "/scvfd/staging/buildings.geojson"
      }
    },
    "layers": [
      {
        "id": "hillshade-layer",
        "type": "raster",
        "source": "hillshade",
        "paint": {
          "raster-opacity": 1,
          "raster-contrast": 0.3
        }
      },
      {
        "id": "roads-layer",
        "type": "line",
        "source": "roads",
        "text-field": [
          "get",
          "name"
        ],
        "paint": {
          "line-color": "rgb(0, 0, 0)",
          "line-width": [
            "match",
            [
              "get",
              "STREETTYPE"
            ],
            "RD",
            5,
            "DR",
            3,
            2
          ]
        }
      },
      {
        "id": "roads-label-layer",
        "type": "symbol",
        "source": "roads",
        "layout": {
          "symbol-placement": "line",
          "text-offset": [
            0,
            2
          ],
          "text-font": [
            "Open Sans Regular"
          ],
          "text-field": [
            "get",
            "STREETNAME"
          ],
          "text-size": 20
        }
      },
      {
        "id": "buildings-layer",
        "type": "fill",
        "source": "buildings",
        "text-field": [
          "get",
          "name"
        ],
        "paint": {
          "fill-color": "rgb(200, 50, 50)"
        }
      }
    ]
  }
});
"""
ht4="""
            const td = new terraDraw.TerraDraw({{
                adapter: new terraDraw.TerraDrawMapLibreGLAdapter({{
                map: map,
                lib: maplibregl,
                }}),
                modes: [new terraDraw.TerraDrawLineStringMode()],
            }});

            // Initialize Terra Draw
            td.start();

            td.setMode("{modestring}");

            // Add save button functionality
            document.getElementById('save-button').addEventListener('click', function() {{
                const features = td.getSnapshot();
                
                {controls_string}
                // const featureText = document.getElementById('feature-text').value;
                // const vecWidth = document.getElementById('{width_att}').value;
                // const features = td.getFeatures();
                
                // Add name to each feature's properties
                features.forEach(feature => {{
                    if (!feature.properties) {{
                        feature.properties = {{}};
                    }}
                    feature.properties.text = featureText;
                    feature.properties.{width_att} = vecWidth;
                
                }});
                const geojson = {{
                    "type": "FeatureCollection",
                    "layer": "{ea['name']}",
                    "features": features
                    }};
                
                for(i = 0; i < features.length; i++){{
                    var xmlhttp = new XMLHttpRequest();   // new HttpRequest instance 
                    xmlhttp.open("POST", 'http://fireatlas.org:9998/store');
                    xmlhttp.setRequestHeader("Content-Type", "application/json");
                    var geojson_data = JSON.stringify({{"data":geojson}});
                    alert(geojson_data);
                    xmlhttp.send(geojson_data);
                    }};
                xmlhttp.onreadystatechange = function() {{
                if (xmlhttp.readyState == 4 && xmlhttp.status == 200) {{
                    alert('upload successful!');
                }} else if (xmlhttp.readyState == 4 && xmlhttp.status !== 200){{
                    alert('looks like something went wrong');
                }}
            }} }} );
            

            </script></body></html>
"""

In [ ]:
grass = '/usr/bin/grass'
sys.path.insert(
    0,subprocess.check_output([grass, "--config", "python_path"], text=True).strip())

my_env = os.environ.copy()
import grass.jupyter as gj
GRASS_LOC = dc['name']
# GRASS_LOC = GRASS_LOC_NAME + datetime.datetime.now().strftime("%I:%M%p_%B-%d-%Y")
session = gj.init("~/grassdata", GRASS_LOC, "PERMANENT")

my_env["PYTHONPATH"] = f"/usr/lib/grass83/etc/python:{my_env['PATH']}"

# TODO: ???
import grass.script as gs
    

In [ ]:
ac['outlets']['geopackage']['layers'] = []

atlas.materialize_outlet(ac, dc, ac['outlets']['geopackage'])

In [ ]:
atlas.grass_location_initialize("scvfd")

In [ ]:
atlas.build_map("scvfd")

In [ ]:
atlas.map_image("scvfd")

In [ ]:
g.crs

g2=g.to_crs("EPSG:4269")

g.explore()

# g.clip(geo_polygon).explore()
g.clip(g3).explore()

import shapely
geo = dc['geometry']['bbox']
geo_polygon = shapely.box(geo['west'], geo['south'], geo['north'], geo['east'])

from geojson import Feature, FeatureCollection,Point, Polygon
import geopandas as gpd

[(geo[x], geo[y]) for x,y in [
                ('west', 'north'), ('east', 'north'), ('east', 'south'), ('west', 'south')]
                 ]

pts=[(geo[x], geo[y]) for x,y in [
                ('west', 'north'), ('east', 'north'), ('east', 'south'), ('west', 'south')]
                 ]
fc=FeatureCollection([Feature(geometry=Polygon([pts]))])
fc

#poly = FeatureCollection([Feature(geometry=Polygon(pts))])
g3=gpd.GeoDataFrame.from_features(fc)

g3.set_crs("EPSG:4326").explore()
# EPSG:4326

g3.set_crs("EPSG:4269").explore()

del(g['quadkey'])

g.to_file("test2.geojson", driver="GeoJSON",crs='EPSG:4269', engine="fiona")

In [ ]:
ac['outlets'].keys()

In [ ]:
ac['eddies']

In [ ]:
bb=dc['geometry']['bbox']
w=(bb['east'] - bb['west'])/4.0


In [ ]:
bb['north'] - 3.0* w

In [ ]:
import matplotlib as mpl
cmap = mpl.colormaps['gist_gray']

In [ ]:
#myc=cmap.copy()

In [ ]:
import numpy as np
f=lambda x: (9.0+x)/10.0
myc = mpl.colors.ListedColormap([[f(r), f(g), f(b), a] for r,g,b,a in cmap(np.linspace(0.0,1.0, 20))])


In [ ]:
myc

In [ ]:
cmap

In [ ]:
dir(cmap)

In [ ]:
math.radians

In [ ]:
def ll2xyz(lat, long,z):
    n = 2**z
    r = math.radians(lat)
    lts = math.log(math.tan(r) + 1.0/math.cos(r))
    x = n*(0.5 + long/360)
    y = n*(1 - lts/math.pi) / 2
    return math.floor(x),math.floor(y),z

In [ ]:
ll2xyz(40.25, -123.92, 12)

In [ ]:
from fastapi import FastAPI

In [ ]:
a = FastAPI()

In [ ]:
dir(a)

In [ ]:
import fastapi
dir(fastapi.logger.logger)
#fastapi.logger.logger.handlers

fastapi.logger.logger.setLevel(0)

https://fireatlas.org/{x}/{y}/{z}.png

https://geojson.io/#new&map=12/40.228/-123.8937

In [ ]:
fastapi.logger.logger.error("HI")

In [ ]:
import logging
dir(logging)